# AstraForge Toolkit — Local API smoke test

Quick sanity checks for the published package against a locally running AstraForge API, plus the same quick-start snippets from the README.

Prereqs:
- Backend running at `http://localhost:8000` (e.g., `make backend-serve`)
- An API key (create via the in-app API Keys screen or `/api/api-keys/`)
- Package installed (`pip install -e ../astraforge-python-package` or from PyPI)
- For the DeepAgent example, set `OPENAI_API_KEY` (or configure your preferred model) so LangChain can call a chat model.


In [ ]:
# Install the local package in editable mode (run from the examples/ folder)
%pip install -e ..

In [ ]:
%pip install python-dotenv

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import os

BASE_URL = "http://localhost:8000/api"
API_KEY = os.getenv("ASTRA_FORGE_API_KEY")

### Create a sandbox-backed DeepAgent (README quick start)

Same as the README example: configure a SandboxBackend, pick a chat model, and build a DeepAgent that can use sandbox tools. Reuses one sandbox session and thread across multiple invocations.


In [ ]:
# Install a model provider helper for the example (skip if already installed)
%pip install langchain-openai

In [ ]:
from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI
from astraforge_toolkit import (
    DeepAgentClient,
    SandboxBackend,
    sandbox_shell,
    sandbox_python_repl,
    sandbox_open_url_with_playwright,
    sandbox_view_image,
)

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
conv = client.create_conversation()
THREAD_ID = conv.conversation_id
SANDBOX_SESSION_ID = conv.sandbox_session_id
print(f"Using sandbox session {SANDBOX_SESSION_ID} for multiple calls")

def backend_factory(rt):
    return SandboxBackend(
        rt,
        base_url=BASE_URL,
        api_key=API_KEY,
        session_id=SANDBOX_SESSION_ID,
        # optional: session_params={"image": "astraforge/codex-cli:latest"},
    )

model = ChatOpenAI(model="gpt-4o-mini")  # or any chat model you prefer
tools = [
    sandbox_shell,
    sandbox_python_repl,
    sandbox_open_url_with_playwright,
    sandbox_view_image,
]

deep_agent = create_deep_agent(
    model=model,
    backend=backend_factory,
    tools=tools,
)

RUN_CONFIG = {
    "thread_id": THREAD_ID,
    "configurable": {"sandbox_session_id": SANDBOX_SESSION_ID},
}


In [ ]:
deep_agent.invoke(
    {"messages": [{"role": "user", "content": "List files in /workspace"}]},
    config=RUN_CONFIG,
)

# Reuse RUN_CONFIG for follow-up calls to keep the same sandbox session and thread ID.


### Call DeepAgent over HTTP (README quick start)

The client hits the REST API directly; it matches the README snippet and streams chunks.


In [ ]:
from astraforge_toolkit import DeepAgentClient

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
conv = client.create_conversation()
print(f"Conversation ID: {conv.conversation_id}")
print(f"Sandbox session ID: {conv.sandbox_session_id}")

for chunk in client.stream_message(conv.conversation_id, "Hello, sandbox!"):
    print(chunk)


### Optional: send a single non-streaming message

Useful if you just want the final response payload.


In [ ]:
response = client.send_message(
    conversation_id=conv.conversation_id,
    messages=[{"role": "user", "content": "Hello from notebook"}],
    stream=False,
)
response
